In [10]:
import sys
sys.path.append('..')
import configparser
import requests
from datetime import datetime
import json
import os
from src.utility import setup_logger, fetch_config
import pandas as pd
import time
from src.api_extarction import GuardianAPI

config, config_path = fetch_config() 
GUARDIAN_API_KEY = config.get('guardian_api', 'api_key')
logger = setup_logger(__name__)

In [11]:
def guardian_search(search_query, from_date, to_date, api_key, max_retries=3, retry_delay=5):
    url = 'http://content.guardianapis.com/search'
    session = requests.Session()  # Using session for connection pooling
    params = {
        'q': search_query,
        'from-date': from_date,
        'to-date': to_date,
        'page-size': 100,
        'api-key': api_key,
        'show-fields': 'headline,byline,sectionName,webPublicationDate'
    }
    all_results = []
    current_page = 1
    total_pages = 1
    retry_count = 0

    while current_page <= total_pages:
        params['page'] = current_page
        response = session.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            all_results.extend(data['response']['results'])
            total_pages = data['response']['pages']
            print(f"Processed page {current_page}/{total_pages}. Total results so far: {len(all_results)}.")
            current_page += 1
            retry_count = 0  # Reset retry count after a successful request
        else:
            print(f"Error {response.status_code}: {response.text}")
            retry_count += 1
            if retry_count <= max_retries:
                time.sleep(retry_delay * retry_count)  # Exponential back-off
            else:
                print("Max retries reached. Stopping.")
                break 

    return all_results

def convert_to_dataframe(api_results):
    try:
        if 'response' in api_results and 'results' in api_results['response']:
            results = api_results['response']['results']
            df = pd.DataFrame(results)
            print(f"Converted data to DataFrame with {len(df)} rows and {len(df.columns)} columns")
            return df
        else:
            print("No results found or incorrect JSON format")
            return pd.DataFrame()
    except Exception as e:
        print(f"Error processing data into DataFrame: {e}")
        return pd.DataFrame()

In [13]:
def convert_to_dataframe(api_results, filter_query="Justin Trudeau"):
    try:
        if 'response' in api_results and 'results' in api_results['response']:
            results = api_results['response']['results']
            
            # Filter results based on the filter_query
            filtered_results = [
                result for result in results
                if filter_query.lower() in result.get('webTitle', '').lower()
                or filter_query.lower() in result.get('fields', {}).get('headline', '').lower()
            ]
            
            df = pd.DataFrame(filtered_results)
            print(f"Converted data to DataFrame with {len(df)} rows and {len(df.columns)} columns")
            print(f"Filtered data for '{filter_query}': {len(df)} rows")
            return df
        else:
            print("No results found or incorrect JSON format")
            return pd.DataFrame()
    except Exception as e:
        print(f"Error processing data into DataFrame: {e}")
        return pd.DataFrame()

In [12]:
search_query = "Justin Trudeau"
from_date = "2018-01-01"
to_date = datetime.now().strftime("%Y-%m-%d")
GUARDIAN_API_KEY = config.get('guardian_api', 'api_key')
api_key = GUARDIAN_API_KEY
articles2 = guardian_search(search_query, from_date, to_date, api_key)

2227ca4a-5a23-4f3d-a35b-c7f28638f4ba

Error 429: {
  "message":"API rate limit exceeded"
}
Error 429: {
  "message":"API rate limit exceeded"
}
Error 429: {
  "message":"API rate limit exceeded"
}
Error 429: {
  "message":"API rate limit exceeded"
}
Max retries reached. Stopping.
